In [11]:
import os
import csv
import sys
import re


import heapq
from collections import defaultdict
from operator import itemgetter
import numpy as np
import pandas as pd

from surprise.model_selection import train_test_split, LeaveOneOut
from surprise import Dataset, Reader, SVD, KNNBaseline, KNNBasic
from collections import defaultdict

# Load data

In [12]:
def loadData(productsPath,userId_, reorder_):
    data = 0
    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(0, 1))
    df = pd.merge(userId_, reorder_, how='inner', on=['order_id', 'order_id'])
    data = Dataset.load_from_df(df[['user_id', 'product_id', 'reordered']], reader)

    with open(productsPath, newline='', encoding='utf-8') as csvfile:
            productsReader = csv.reader(csvfile)
            next(productsReader)  #Skip header line
            for row in productsReader:
                productID = int(row[0])
                productName = row[1]
                productID_to_name[productID] = productName
                name_to_productID[productName] = productID

    return data

In [13]:
productsPath = './dataset/products.csv'
reorder_ = pd.read_csv('./dataset/order_products__prior.csv')
userId_ = pd.read_csv('./dataset/orders.csv')

productID_to_name = {}
name_to_productID = {}
data = loadData(productsPath,userId_, reorder_)

# Train Model

In [ ]:
trainSet = data.build_full_trainset()
sim_options = {'name': 'cosine','user_based': True}

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

# Get top N similar users to our test subject

In [ ]:
testSubject = '85'
k = 10

testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )

kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

# Get the stuff they rated, and add up ratings for each item, weighted by user similarity

In [ ]:
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

# Build a dictionary of stuff the user has already seen

In [ ]:
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

# Get top-rated items from similar users:

In [ ]:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break